In [1]:
import os
import datetime
import torch
import random
import pickle
import pandas as pd

#os.chdir(os.path.join(os.getcwd(), 'LAS Model'))
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from data import SpeechDataset, AudioDataLoader
from listener import Listener
from attend_and_spell import AttendAndSpell
from seq2seq import Seq2Seq
from utils import  train
from torchviz import make_dot

ModuleNotFoundError: No module named 'data'

In [2]:
def get_chars(lang, save_file=None, train_df=None):
    if lang=='eng':
        chars = ['<sos>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', \
                'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', \
                'y', 'z', ' ', "'", '<eos>', '<pad>']
    elif lang=='chinese':            
        try:
            with open(save_file, 'rb') as f:
                chars = pickle.load(f) # load file
        except FileNotFoundError:
            chars = [' ', '<sos>']
            for idx in range(train_df.shape[0]):
                _, sent = train_df.iloc[idx]
                for c in sent:
                    if c not in chars:
                        chars.append(c)
            chars = chars + ['<eos>', '<pad>', '<unk>']
            with open(save_file, 'wb') as f:
                pickle.dump(chars, f) # save file
    else:
        raise NotImplementedError
    
    print('Number of chars', len(chars))
    return chars


# Used when each sentence is in a separate text file
def make_train_df(dataset_dir):
    data = []
    files = os.listdir(dataset_dir)
    for f in files:
        if '.txt' in f:
            with open(os.path.join(dataset_dir, f), 'r') as text:
                data.append((f.replace('.txt', ''), text.readline()))
                
    train_df = pd.DataFrame(data, columns=['id', 'sent'])
    train_df.to_csv(os.path.join(dataset_dir, 'train_df.csv'), header=None)
    print(train_df.head())


# Used for ai_shell dataset, when all sentences are in a single text file
def read_transcrpt(transcript_dir):
    transcript_dir = '../../../Dataset'
    with open(os.path.join(transcript_dir, 'aishell_transcript_v0.8.txt')) as f:
        data_list = f.readlines()

    data = []
    for example in data_list:
        id_, sent = str(example.split(' ')[0]), str(' '.join(example.split(' ')[1:-1])) # -1 to remove '\n'
        data.append((id_, sent))

    print('Num examples:', len(data))
    data_df = pd.DataFrame(data, columns=['id', 'sent'])
    data_df.to_csv(os.path.join(transcript_dir, 'train_df.csv'))
    data_df.head()

In [10]:
def decode_pred_sent(out):
    pred_sent = []
    for t in out:
        lol = t.max(dim=1)[1].item()
        pred_sent.append(token_to_char[lol])
    return ''.join(pred_sent)

def decode_true_sent(y):
    sent = []
    for t in y:
        sent.append(token_to_char[t.item()])
    return ''.join(sent)

DEVICE : cuda


FileNotFoundError: [Errno 2] File b'../../../Dataset/data_aishell/train_df.csv' does not exist: b'../../../Dataset/data_aishell/train_df.csv'

In [13]:
num_sent = 10
model.eval()
model.tf_ratio = 0.9

for _ in range(num_sent):
    
    idx = random.randint(0, train_df.shape[0])
    trial_dataset = SpeechDataset(train_df, dataset_dir, sos_token, char_to_token, eos_token)

    x, y = trial_dataset.__getitem__(idx)
    # plt.imshow(x[0,:,:].detach())

    # Model output
    target = y.unsqueeze(dim=0).to(DEVICE)
    data = x.permute(0, 2, 1).to(DEVICE)
    loss, output = model(data, target)
    print("True sent : ", decode_true_sent(y))
    print("Pred sent : ", decode_pred_sent(output))
    print("Loss :", loss.item())    
    print("\n")

tensor([   1, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255,
        4255, 4255, 4255, 4255, 4255, 4255, 4255, 4253])
True sent :  <sos><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><eos>
Pred sent :  <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
Loss : 7.456485748291016


tensor([   1, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255,
        4253])
True sent :  <sos><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><eos>
Pred sent :  <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
Loss : 7.510329246520996


tensor([   1, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255, 4255,
        4253])
True sent :  <sos><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><eos>
Pred sent :  <unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
Loss : 7.510329723358154


tensor([   1, 4255, 4255, 4255, 4255, 4255, 4255